# Jazz melody generation using LSTMs

*** NOTE: This does the same thing as the [`melody_lstm.ipynb`](melody_lstm.ipynb) notebook, except that here we use the `jazzaiexperiments` module instead of defining everything inside the notebook itself. ***

First, let's import the `jazzaiexperiments` module:

In [1]:
import jazzaiexperiments

Using TensorFlow backend.


Next, we'll pick a tune:

In [2]:
tune_name = "ColemanHawkins_BodyAndSoul_FINAL"

Let's build and train an LSTM model from a MIDI file:

In [3]:
model, note_events, input_filepath = jazzaiexperiments.lstm.train_on_midi_input(tune_name,
                                                                                "../data/midi/quantized/",
                                                                                "../data/models/",
                                                                                num_epochs=1)
model.summary()

Created note events from ../data/midi/quantized/ColemanHawkins_BodyAndSoul_FINAL.mid
Formatted note data (625 seqs of length 10, 365 unique notes)
Created model
Epoch 1/1
625/625 [==============================] - 4s - loss: 5.8943     
Trained model over 1 epochs
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 256)           264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 365)               93805     
Total par

This is some example output for 10 epochs (it's not very good, and basically gets stuck in a two-to-three-note loop): https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-single-melody-coleman-hawkins-body-and-soul-10-epochs

We've actually trained this model before, so let's load some existing weights (from 100 epochs of training) into our current model:

In [4]:
model = jazzaiexperiments.lstm.load_model_weights(model, "../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170702124647869274_99_0.8517.hdf5")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 256)           264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 365)               93805     
Total params: 883,309
Trainable params: 883,309
Non-trainable params: 0
_________________________________________________________________


Note that we can also do this by passing in a `weights_filepath` argument into the `lstm_train_on_midi_input` function when creating the model.

And let's generate some output!

In [5]:
notes_out = jazzaiexperiments.lstm.generate_midi_output(model, note_events,
                                                        num_notes_to_generate=100,
                                                        random_seed=False,
                                                        add_seed_to_output=True,
                                                        tune_name=tune_name,
                                                        midi_source_filepath=input_filepath,
                                                        data_dir="../data/output")
notes_out[:20]

Constructed input sequence: [47, 41, 43, 45, 74, 109, 80, 98, 78, 42]
Generated 100 notes
Wrote to MIDI file at ../data/output/out_ColemanHawkins_BodyAndSoul_FINAL_20170705204606246541.mid


[(51, 100, 76230, 16170),
 (51, 90, 0, 9240),
 (51, 90, 0, 43890),
 (51, 100, 0, 6930),
 (53, 90, 0, 6930),
 (54, 110, 0, 6930),
 (53, 100, 0, 4620),
 (54, 100, 0, 4620),
 (53, 90, 0, 39270),
 (51, 90, 0, 13860),
 (58, 100, 0, 13860),
 (58, 100, 0, 20790),
 (58, 90, 0, 48510),
 (56, 90, 0, 27720),
 (57, 100, 0, 6930),
 (50, 80, 0, 6930),
 (56, 90, 0, 13860),
 (49, 80, 0, 13860),
 (58, 110, 27720, 9240),
 (56, 100, 0, 9240)]

Example output for 100 epochs: https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-single-melody-coleman-hawkins-body-and-soul-100-epochs